In [19]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import sys
from sklearn.model_selection import train_test_split
sys.path.append("..")
from scipy.stats import kurtosis
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report,accuracy_score,roc_auc_score

In [10]:
#Handling with zero values or null data:
df = pd.read_csv('TrainingData.csv', delimiter=',', skip_blank_lines=True)
df=df.drop(['Timestamp'],axis=1)
df = df.dropna(axis=1, how='all')
df=df.sample(frac=1)
df = df.replace('-',value=np.NaN)
df = df.fillna(method='ffill')
df = df.fillna(method='bfill')
target = df[['TargetVariable']]
my_data = df.drop(['TargetVariable'], axis=1)

/root/miniconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (1,2,3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [11]:
x_train,x_valid,y_train,y_valid=train_test_split(my_data,target,test_size=0.2,shuffle=False)

In [12]:
#Select features using mutual information:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif
Model = SelectKBest(mutual_info_classif, k=10).fit(x_train,y_train)
X_train = Model.transform(x_train)
X_valid = Model.transform(x_valid)
print(X_train.shape)
print(X_valid.shape)

/root/miniconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


(4737, 10)
(1185, 10)


In [13]:
#Select features based on importances using Decision tree classifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
clf = ExtraTreesClassifier()
clf = clf.fit(x_train, y_train)
clf.feature_importances_  
model = SelectFromModel(clf, prefit=True)
X_train = model.transform(x_train)
X_valid= model.transform(x_valid)
print(X_train.shape)
print(X_valid.shape)

/root/miniconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
/root/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


(4737, 325)
(1185, 325)


In [33]:
#With variable selection
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model = model.fit(X_train, y_train)
y_predicted=model.predict(X_valid)
props=model.predict_proba(X_valid)[:,1]
auc=roc_auc_score(y_valid,props)
print('Logistic regression Acc:',accuracy_score(y_valid,y_predicted))
print('Logistic regression AUC :',auc)
print('Mean accuracy:',model.score(X_valid,y_valid))

/root/miniconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Logistic regression Acc: 0.7029535864978903
Logistic regression AUC : 0.752005626482614
Mean accuracy: 0.7029535864978903


In [34]:
#Logistic regression without variable selection
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model = model.fit(x_train, y_train)
y_predicted=model.predict(x_valid)
props=model.predict_proba(x_valid)[:,1]
model.score(x_valid,y_valid)
print('Logistic regression Acc:',accuracy_score(y_valid,y_predicted))
print('Logistic regression AUC :',auc)
print('Mean accuracy:',model.score(x_valid,y_valid))

/root/miniconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Logistic regression Acc: 0.7088607594936709
Logistic regression AUC : 0.752005626482614
Mean accuracy: 0.7088607594936709


In [26]:
from sklearn import svm
'SVM Classifier with variable selection'
SVC=svm.SVC(probability=True)
SVC=SVC.fit(X_train,y_train)
y_predicted=SVC.predict(X_valid)
props=SVC.predict_proba(X_valid)[:,1]
auc=roc_auc_score(y_valid,props)
print('SVM Acc:',accuracy_score(y_valid,y_predicted))
print('SVM AUC:',auc)
print('Mean accuracy:',SVC.score(X_valid,y_valid))

/root/miniconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVM Acc: 0.7223628691983123
SVM AUC: 0.8119598517088057
Mean accuracy: 0.7223628691983123


In [27]:
'SVM classifiier without variable selection'
SVC=SVC.fit(x_train,y_train)
y_predicted=SVC.predict(x_valid)
props=SVC.predict_proba(x_valid)[:,1]
auc=roc_auc_score(y_valid,props)
print('SVM Acc:',accuracy_score(y_valid,y_predicted))
print('SVM AUC:',auc)
print('Mean accuracy:',SVC.score(x_valid,y_valid))

/root/miniconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVM Acc: 0.7071729957805907
SVM AUC: 0.8005563899915364
Mean accuracy: 0.7071729957805907
